In [1]:
import pandas as pd
import dash
#import dash_html_components as html
from dash import html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
print(spacex_df["Launch Site"].unique())
print(spacex_df.keys())

['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']
Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')


In [3]:

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                 dcc.Dropdown(id='site-dropdown',
                                            options=[{'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'site1', 'value': 'CCAFS LC-40'},
                                                    {'label': 'site2', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'site3', 'value': 'KSC LC-39A'},
                                                    {'label': 'site4', 'value': 'CCAFS SLC-40'}
                                                    ],   
                                             value ='ALL', # 'CCAFS LC-40',#{'label': 'site1', 'value': 'CCAFS LC-40'},#{'label': 'All Sites', 'value': 'ALL'} ,
                                             placeholder="Select a Launch Site here",
                                              searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10_000,
                                                step = 1000,
                                                value=[0,10_000],
                                                marks = {0:"0",2000:"2000",
                                                         4000:"4000",6000:"6000",
                                                         8000:"8000",10_000:"10.000"}
                                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df["class"]==1]
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, #values='class', 
        names="Launch Site",#'pie chart names', 
        title='All the successfull launches spread across sites')
        return fig
    else:
        filtered_df = spacex_df[spacex_df["Launch Site"]== entered_site]
        fig = px.pie(filtered_df, #values=["succes","failure"],
                    names = "class",
                     #labels=["Success","Faiulre"],#{1:"Succes",0:"Failure"},
                    #color_discrete_map={""},
                    title = "succes and failure of site %s" %entered_site)

        return fig
        # return the outcomes piechart for a selected site
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs,
#`success-payload-scatter-chart` as output

@app.callback(
    Output(component_id="success-payload-scatter-chart",component_property="figure"),
            [Input(component_id="site-dropdown",component_property="value"),
             Input(component_id="payload-slider",component_property="value")]
            
            )
def scatter(site,slider_range):
    low, high = slider_range
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)

    if site == "ALL":
        fig = px.scatter(spacex_df[mask],
                         x='Payload Mass (kg)',
                         y='class',
                         color="Booster Version Category")
        return fig
    else:
        filtered_df  =spacex_df[spacex_df["Launch Site"]==site]
        fig = px.scatter(filtered_df[mask],
                         x='Payload Mass (kg)',
                         y='class',
                         color="Booster Version Category")
        return fig
    

# Run the app
if __name__ == '__main__':
    app.run_server()